In [10]:
#class for getting & parsing RSS feeds
import feedparser as fp
import pandas as pd
import html
from nltk.tokenize import TreebankWordTokenizer,WhitespaceTokenizer
import string
from nltk.corpus import stopwords
import re
import time
from guess_language import guess_language
import time
import numpy as np
from website.code_bin import Cleaner

Now that we have scraped iTunes for podcast metadata, we want to get podcast episode data.

To do so, we'll scrape directly from each podcasts' RSS feed.

In [11]:
#Load in putative podcasts, turn it into a dataframe. 
import pickle
import pandas as pd


#load in itunes_request_db
floc = '/home/bmassi/Dropbox/professional/Insight/data/'
with open(floc+'raw_itunes_requests1528609065.pkl','rb') as fid:
    raw_itunes_requests = pickle.load(fid)
    
#turn everything into a pandas dataframe
formatted_results = []
bads = []
cnames = ['']
for rir in raw_itunes_requests:
    for p in rir.json()['results']:
        if(p['kind']=='podcast'):
            formatted_results.append(p)

podcast_df = pd.DataFrame(formatted_results)
podcast_df = podcast_df.loc[podcast_df['feedUrl'].isnull()==False]

In [12]:
#filter out podcasts that weren't updated recently (look at fraction of remaining dataset as a function of cutoff)
from datetime import datetime,timedelta

#parameters for analysis
target_cutoff = 45#filter out podcasts that weren't updated for this many days
comparator_day = datetime(2018,6,11,14,5,23,424906)#This was "today" on Monday 6-11-18

podcast_df['releaseDate'] = pd.to_datetime(podcast_df['releaseDate'])
podcast_df = podcast_df.loc[podcast_df['releaseDate'] > (comparator_day - timedelta(days=target_cutoff))]

print(len(podcast_df))

137785


In [13]:
#Crawl & scrape RSS feeds
#We will visit every RSS feed and scrape all the episode data.
import numpy as np
import time
import socket
import feedparser as fp
import requests

MAX_REQUEST_DURATION = 10 #seconds
socket.setdefaulttimeout(MAX_REQUEST_DURATION)

step_size = 500#number of feeds to get/save at once
foutloc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries2/'
crawler = FeedCrawler(podcast_df,foutloc)

try:
    crawler.loadcounters()
    print('initialized! counter = %d,filecounter = %d,id = %s' %
          (crawler.counter,crawler.filecounter,crawler.identifier))
except:
    print("could not initialize")
    init = 0

flag = 1
while(flag):
    start_time = time.time()
    try:
        crawler.getFeed(step_size)
        crawler.parseFeeds()
        crawler.cleanFeeds()
        crawler.save()
    except NoMoreFeedError as e:
        flag = 0
        print("Job's done!")
    stop_time = time.time()
    duration = stop_time - start_time
    print('Counter=%d/%d (duration=%.2f, step=%d)' % (crawler.counter,crawler.getNcasts(),duration,step_size))

initialized! counter = 500,filecounter = 1,id = 1529184593.957413
Parsed!
Clean!
Saving 340 feeds...
Counter=1000/137785 (duration=345.92, step=500)
Parsed!
Clean!
Saving 365 feeds...
Counter=1500/137785 (duration=353.64, step=500)
Parsed!
Clean!
Saving 322 feeds...
Counter=2000/137785 (duration=392.31, step=500)
Parsed!
Clean!
Saving 348 feeds...
Counter=2500/137785 (duration=369.92, step=500)
Parsed!
Clean!
Saving 306 feeds...
Counter=3000/137785 (duration=399.55, step=500)
Parsed!
Clean!
Saving 291 feeds...
Counter=3500/137785 (duration=348.34, step=500)
Parsed!
Clean!
Saving 303 feeds...
Counter=4000/137785 (duration=383.37, step=500)
Parsed!
Clean!
Saving 328 feeds...
Counter=4500/137785 (duration=403.08, step=500)
Parsed!
Clean!
Saving 328 feeds...
Counter=5000/137785 (duration=387.65, step=500)
Parsed!
Clean!
Saving 349 feeds...
Counter=5500/137785 (duration=326.87, step=500)
Parsed!
Clean!
Saving 259 feeds...
Counter=6000/137785 (duration=416.46, step=500)
Parsed!
Clean!
Saving

Parsed!
Clean!
Saving 289 feeds...
Counter=49500/137785 (duration=440.29, step=500)
Parsed!
Clean!
Saving 292 feeds...
Counter=50000/137785 (duration=416.34, step=500)
Parsed!
Clean!
Saving 334 feeds...
Counter=50500/137785 (duration=443.57, step=500)
Parsed!
Clean!
Saving 283 feeds...
Counter=51000/137785 (duration=442.05, step=500)
Parsed!
Clean!
Saving 322 feeds...
Counter=51500/137785 (duration=419.66, step=500)
Parsed!
Clean!
Saving 349 feeds...
Counter=52000/137785 (duration=436.68, step=500)
Parsed!
Clean!
Saving 351 feeds...
Counter=52500/137785 (duration=385.56, step=500)
Parsed!
Clean!
Saving 325 feeds...
Counter=53000/137785 (duration=363.16, step=500)
Parsed!
Clean!
Saving 223 feeds...
Counter=53500/137785 (duration=328.12, step=500)
Parsed!
Clean!
Saving 287 feeds...
Counter=54000/137785 (duration=380.56, step=500)
Parsed!
Clean!
Saving 275 feeds...
Counter=54500/137785 (duration=409.63, step=500)
Parsed!
Clean!
Saving 291 feeds...
Counter=55000/137785 (duration=383.03, st

Parsed!
Clean!
Saving 206 feeds...
Counter=98500/137785 (duration=544.34, step=500)
Parsed!
Clean!
Saving 196 feeds...
Counter=99000/137785 (duration=456.42, step=500)
Parsed!
Clean!
Saving 208 feeds...
Counter=99500/137785 (duration=520.16, step=500)
Parsed!
Clean!
Saving 216 feeds...
Counter=100000/137785 (duration=561.90, step=500)
Parsed!
Clean!
Saving 123 feeds...
Counter=100500/137785 (duration=451.02, step=500)
Parsed!
Clean!
Saving 241 feeds...
Counter=101000/137785 (duration=685.27, step=500)
Parsed!
Clean!
Saving 200 feeds...
Counter=101500/137785 (duration=596.45, step=500)
Parsed!
Clean!
Saving 257 feeds...
Counter=102000/137785 (duration=496.56, step=500)
Parsed!
Clean!
Saving 227 feeds...
Counter=102500/137785 (duration=477.70, step=500)
Parsed!
Clean!
Saving 187 feeds...
Counter=103000/137785 (duration=551.27, step=500)
Parsed!
Clean!
Saving 201 feeds...
Counter=103500/137785 (duration=556.47, step=500)
Parsed!
Clean!
Saving 222 feeds...
Counter=104000/137785 (duration=5

IndexError: single positional indexer is out-of-bounds

In [3]:
#Save out the scraped RSS feeds.
import pickle

floc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries2/'
with open(floc+'feeds_0_1529184593.957413.pkl','rb') as fid:
    prog = pickle.load(fid)


2